In [1]:
# Step 1: Load GROQ API Key
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
groq_api_key = os.environ["GROQ_API_KEY"]

In [5]:
# Step 2: Import LLM (Groq)
from langchain_groq import ChatGroq

model = ChatGroq(api_key="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", model="qwen/qwen3-32b")

In [6]:
# Step 3: Basic LCEL Chain
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("tell me a curious fact about {soccer_player}")
output_parser = StrOutputParser()

chain = prompt | model | output_parser
chain.invoke({"soccer_player": "Messi"})

'<think>\nOkay, the user wants a curious fact about Messi. Let me think about what I know about him. He\'s a famous soccer player, from Argentina, plays for PSG now, maybe? But I need to check if that\'s current. Also, his achievements, like Ballon d\'Ors. Wait, I remember he won 8 Ballon d\'Ors, which is a lot. But is there something curious about that? Maybe the fact that he\'s the only one to win it 8 times. That\'s a fact, but maybe not the most curious.\n\nWhat about his family? Does he have any siblings with interesting careers? His brother is a goalkeeper, maybe? Or maybe his nickname? Leo Messi, the nickname is Leo, but that\'s common. Wait, his nickname in Argentina is Leo, but his full name is Lionel Andrés. Maybe something about his early career? Like he was diagnosed with a growth hormone deficiency as a child. That required treatment, which his family couldn\'t afford, and Barcelona paid for it when he moved there. That\'s a common fact, but is it curious? Maybe, because i

In [7]:
# Step 4: Using `.bind()` to stop generation after "Ronaldo"
chain = prompt | model.bind(stop=["Messi"]) | output_parser
chain.invoke({"soccer_player": "Messi"})

'<think>\nOkay, so I need to find a curious fact about '

## ***Combining LCEL Chains***

In [8]:
prompt = ChatPromptTemplate.from_template("tell me a sentence about {politician}")
chain = prompt | model | StrOutputParser()
chain.invoke("Chamberlain")

"<think>\nOkay, the user wants a sentence about Chamberlain. Let me figure out which Chamberlain they're referring to. There are a few notable ones.\n\nFirst, there's Neville Chamberlain, the British Prime Minister during the 1930s known for appeasing Hitler before WWII. That's a common reference. Then there's Oliver Chamberlain, the American football player, and maybe others like John Chamberlain, the artist. But since the user didn't specify, I should go with the most well-known one, which is likely Neville Chamberlain.\n\nNow, I need to create a sentence that's informative but concise. Maybe mention his role in the Munich Agreement and the policy of appeasement. Let me check the key points: he was the Prime Minister, he signed the Munich Agreement in 1938, which allowed Germany to annex the Sudetenland, and this policy is often seen as a failure because it didn't stop Hitler. \n\nI should make sure the sentence is accurate. Let me structure it: Neville Chamberlain, as British Prime 

## ***Combined Chain***

In [9]:
historian_prompt = ChatPromptTemplate.from_template("Was {politician} positive for Humanity?")
composed_chain = {"politician": chain} | historian_prompt | model | StrOutputParser()
composed_chain.invoke({"politician": "Lincoln"})

'<think>\nOkay, the user asked for a sentence about Lincoln, and I provided one. Now they\'re asking "Was positive for Humanity?" Hmm, maybe they\'re questioning if Lincoln\'s impact was positive. I need to address that.\n\nFirst, I should confirm that Lincoln\'s actions were indeed beneficial. He ended slavery, preserved the Union, and his leadership during the Civil War had long-term positive effects. But I should also consider any counterarguments. Some might argue about his policies towards Native Americans or the expansion of federal power, but those are more nuanced points.\n\nThe user\'s question seems to be seeking validation of Lincoln\'s positive contributions. I should clearly state that his actions were positive, especially regarding ending slavery and preserving the Union. Mention specific achievements like the Emancipation Proclamation and the Gettysburg Address to reinforce that. Also, highlight his moral stance against slavery as a key factor. Make sure to keep it conci

## ***Chain Inside Another Chain***

In [10]:
from operator import itemgetter

prompt1 = ChatPromptTemplate.from_template("what is the country {politician} is from?")
prompt2 = ChatPromptTemplate.from_template("what continent is the country {country} in? respond in {language}")

chain1 = prompt1 | model | StrOutputParser()

chain2 = (
    {"country": chain1, "language": itemgetter("language")}
    | prompt2
    | model
    | StrOutputParser()
)

chain2.invoke({"politician": "Miterrand", "language": "French"})

"<think>\nOkay, the user asked about the continent of the country where Mitterrand is from. I need to figure out which Mitterrand they're referring to. The most likely is François Mitterrand, the former French president. He was born in France, so the country is France. Now, France is in Europe. I should confirm that there's no other Mitterrand from a different country, but I can't think of any. So the answer is Europe. I'll make sure to mention François Mitterrand to avoid confusion with any other namesakes.\n</think>\n\nFrançois Mitterrand était un homme politique français, ancien président de la République (1981-1995) et Premier ministre (1959-1962). Il est né en France, pays situé en **Europe**. Ainsi, le continent associé à son pays d'origine est **l'Europe**."

## ***LCEL Chain in a RAG Pipeline***

In [11]:
import bs4
from langchain import hub
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_text_splitters import RecursiveCharacterTextSplitter

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [12]:
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)

In [13]:
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

In [14]:
from langchain.embeddings import HuggingFaceEmbeddings

embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

C:\Users\emon1\AppData\Local\Temp\ipykernel_2580\1728591684.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
c:\Users\emon1\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [15]:
vectorstore = Chroma.from_documents(documents=splits, embedding=embedding_model)
retriever = vectorstore.as_retriever()

In [16]:
prompt = hub.pull("rlm/rag-prompt")

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

c:\Users\emon1\AppData\Local\Programs\Python\Python311\Lib\site-packages\langsmith\client.py:241: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [17]:
rag_chain.invoke("What is Task Decomposition?")

'<think>\nOkay, the user is asking about Task Decomposition. Let me look at the context provided.\n\nFrom the context, task decomposition can be done in three ways: using LLM with prompts like "Steps for XYZ," task-specific instructions, or human inputs. Also mentioned is the LLM+P approach where a classical planner is used with PDDL. There\'s a mention of Chain of Thought (CoT) and Tree of Thoughts (ToT) methods which break tasks into steps and explore multiple reasoning paths.\n\nI need to define Task Decomposition concisely. The answer should mention breaking down tasks into subgoals or steps, the methods like prompting or planning, and maybe the purpose like making complex tasks manageable. The user wants three sentences max, so I need to be concise. Let me check the key points again. Avoid mentioning challenges unless necessary, focus on the definition and methods. Make sure to reference the context accurately without adding extra info.\n</think>\n\nTask Decomposition is the proce

In [18]:
rag_chain.invoke("What is Self-Reflection?")

'<think>\nOkay, the user is asking what self-reflection is. Let me look at the provided context. The context mentions the Reflexion framework, which uses self-reflection for agents. It says self-reflection is created by showing two-shot examples of failed trajectories and ideal reflections. These reflections are added to the agent\'s working memory. Also, there\'s a part about ReAct integrating reasoning and acting, but the key part here is the Reflexion framework.\n\nSelf-reflection in this context seems to be a method where agents learn from their mistakes by analyzing failed attempts and using that to adjust future actions. The examples provided include failed trajectories and ideal reflections, which guide the agent. The agent uses these reflections in its memory to improve. The Reflexion framework uses a heuristic function to detect inefficiency or hallucination, leading the agent to reset and start over if needed.\n\nI need to make sure I\'m not confusing self-reflection with ref